In [ ]:
# %%
#! python3
#
# Lending Club Case Study
# In this case study, we will use EDA to understand how consumer attributes and loan attributes influence the tendency of default.
#
#
#

In [2]:
# Import the modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [48]:
# Set the data file name
data_file_name = "loan.csv"
raw_data_df = pd.read_csv(data_file_name, low_memory=False)

raw_data_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [49]:
# Check the number of rows and columns
raw_data_df.shape

(39717, 111)

In [50]:
raw_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


In [51]:
# Clean data for analysis
raw_data_df["term"] = raw_data_df["term"].str.replace("months", "")
raw_data_df["int_rate"] = raw_data_df["int_rate"].str.replace("%", "")
raw_data_df["revol_util"] = raw_data_df["revol_util"].str.replace("%", "")
raw_data_df["emp_length"] = raw_data_df["emp_length"].str.replace("< ", "").str.replace("+ years","").str.replace(" years","").str.replace("year","")
raw_data_df["emp_length"] = raw_data_df["emp_length"].str.replace("+s", "").str.replace("s","")
raw_data_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [52]:
junk_char = ['+',' ']
raw_data_df[raw_data_df["emp_length"].isin(junk_char)]

raw_data_df["emp_length"].str.find("+")

raw_data_df["emp_length"]

0        10
1         1
2        10
3        10
4         1
         ..
39712     4
39713     3
39714     1
39715     1
39716     1
Name: emp_length, Length: 39717, dtype: object

In [53]:
# Convert the cleaned up columns into numbers
raw_data_df.term = raw_data_df.term.astype(float)
raw_data_df.int_rate = raw_data_df.int_rate.astype(float)
raw_data_df.revol_util = raw_data_df.revol_util.astype(float)
raw_data_df.emp_length = raw_data_df.emp_length.astype(float)


In [54]:
# Get the non numeric values in the Runs column
is_non_numeric = pd.to_numeric(raw_data_df['annual_inc'], errors='coerce').isnull() # errors=’coerce’ will replace all non-numeric values with NaN.
unique_non_numeric_values = raw_data_df[is_non_numeric]['annual_inc'].unique()
unique_non_numeric_values

array([], dtype=float64)

In [56]:
# Get descriptive statistics of key columns
raw_data_df[["loan_amnt", "funded_amnt", "term", "int_rate", "last_pymnt_amnt", "emp_length", "annual_inc"]].describe()


,loan_amnt,funded_amnt,term,int_rate,last_pymnt_amnt,emp_length,annual_inc
count,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,38642.000000,3.971700e+04
mean,11219.443815,10947.713196,42.418007,12.021177,2678.826162,5.092050,6.896893e+04
std,7456.670694,7187.238670,10.622815,3.724825,4447.136012,3.408338,6.379377e+04
min,500.000000,500.000000,36.000000,5.420000,0.000000,1.000000,4.000000e+03
25%,5500.000000,5400.000000,36.000000,9.250000,218.680000,2.000000,4.040400e+04
50%,10000.000000,9600.000000,36.000000,11.860000,546.140000,4.000000,5.900000e+04
75%,15000.000000,15000.000000,60.000000,14.590000,3293.160000,9.000000,8.230000e+04
max,35000.000000,35000.000000,60.000000,24.590000,36115.200000,10.000000,6.000000e+06


In [59]:
# Carve out the defaulters
default_df = raw_data_df[raw_data_df["loan_status"].isin(["Charged Off"])]

default_df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
1,1077430,1314167,2500,2500,2500.0,60.0,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
8,1071795,1306957,5600,5600,5600.0,60.0,21.28,152.39,F,F2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
9,1071570,1306721,5375,5375,5350.0,60.0,12.69,121.45,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
12,1064687,1298717,9000,9000,9000.0,36.0,13.49,305.38,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
14,1069057,1303503,10000,10000,10000.0,36.0,10.65,325.74,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39667,118823,118026,2500,2500,675.0,36.0,12.80,84.00,D,D4,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
39668,118533,117783,2500,2500,825.0,36.0,9.64,80.26,B,B4,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
39669,118523,118519,6500,6500,225.0,36.0,15.01,225.37,F,F1,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
39678,113179,113093,1000,1000,950.0,36.0,10.59,32.55,C,C2,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
